![image-3.png](https://github.com/cafawo/Derivatives/blob/main/figures/fscampus_small.png?raw=1)

# Derivatives

[**Prof. Dr. Fabian Woebbeking**](https://woebbeking.info/)</br>
Assistant Professor of Financial Economics</br>

## Case 4: Bund future and clearing mechanics

#### Task: Calculate the bund future cash flows.

At an interest rate of 8% you enter into a long position of 2 Bund future contracts. The interest rate declines to 7% and you close the contract. Explain the resulting cash flows with an initial margin of 2000 € per contract. (The notional bond is the cheapest to deliver, the yield curve is flat.)

In [14]:
import numpy as np

def bond_price(ytm:float, cpn:float=0.06, maturity:int=10):
    return cpn*sum(1/(1+ytm)**np.array(range(1, maturity+1))) + 1/(1+ytm)**maturity


initial_margin = 4000
print('#'*79)
print(f"t0: initial margin = EUR {-initial_margin:,.2f}")
print(f"t0: pnl            = EUR {0:,.2f}")
print(f"t1: total          = EUR {- initial_margin:,.2f}")

# Bund future tick size is 0.01 (price), hence, round:
delta_mtm = round(bond_price(0.07) - bond_price(0.08), 4)  # New minus old price
delta_position = delta_mtm * 100000 * 2

print('#'*79)
print(f"t1: initial margin = EUR {initial_margin:,.2f}")
print(f"t1: pnl            = EUR {delta_position:,.2f}")
print(f"t1: total          = EUR {delta_position + initial_margin:,.2f}")

###############################################################################
t0: initial margin = EUR -4,000.00
t0: pnl            = EUR 0.00
t1: total          = EUR -4,000.00
###############################################################################
t1: initial margin = EUR 4,000.00
t1: pnl            = EUR 12,800.00
t1: total          = EUR 16,800.00


#### Task: Calculate the settlement payment.

Zock is short a March 04 Future, The EDSP is 116,80 on 8.3.2004. Zock delivers on the 2004-03-10 the 4,5% Bundesanleihe with the maturity 2013-01-04. What is the payment he receives?

Hint: You will need the `class Bond():` (see derivatives.ipynb) or a similar bond pricing software to solve this and the following tasks.

In [15]:
import calendar
from datetime import datetime
from dateutil.relativedelta import relativedelta

class Bond():
    def __init__(self, settlement:str, maturity:str, coupon:float=0):
        self.settlement = datetime.fromisoformat(settlement)
        self.maturity = datetime.fromisoformat(maturity)
        self.coupon = coupon
        if self.coupon > 0:
            # List of coupon payment days (ignoring modified following)
            self.coupon_shedule = [self.maturity]
            while True:
                coupon_date = min(self.coupon_shedule) - relativedelta(years=1)
                if coupon_date < self.settlement: break
                self.coupon_shedule.append(coupon_date)
            self.coupon_shedule.sort()
            # The next element in the coupon shedule (accrued interest)
            self.next_coupon = min(self.coupon_shedule)
            self.previous_coupon = min(self.coupon_shedule) - relativedelta(years=1)
            self.last_coupon = max(self.coupon_shedule)
            # Accrued interest
            self.accrued_period = self.actact(self.previous_coupon, self.settlement)
            self.accrued = self.coupon * self.accrued_period
            
    def actact(self, start_date:datetime, end_date:datetime):
        if start_date == end_date:
            return 0.0
        start_date = datetime.combine(start_date, datetime.min.time())
        end_date = datetime.combine(end_date, datetime.min.time())
        start_year = start_date.year
        end_year = end_date.year
        year_1_diff = 366 if calendar.isleap(start_year) else 365
        year_2_diff = 366 if calendar.isleap(end_year) else 365
        total_sum = end_year - start_year - 1
        diff_first = datetime(start_year + 1, 1, 1) - start_date
        total_sum += diff_first.days / year_1_diff
        diff_second = end_date - datetime(end_year, 1, 1)
        total_sum += diff_second.days / year_2_diff
        return total_sum

    def cleanprice(self, ytm:float=None):
        price = 0
        if self.coupon > 0:
            for t in bond.coupon_shedule:
                t_period = self.actact(self.settlement, t)
                price += self.coupon * 1 / (1 + ytm) ** t_period
            # Dirty price
            price += 100 / (1 + ytm) ** t_period
            # Clean price
            return price - self.accrued
        else:
            price += 100 / (1 + ytm) ** self.actact(self.settlement, self.maturity)
            return price
    
    def dirtyprice(self, ytm:float=None):
        if self.coupon > 0:
            return self.cleanprice(ytm) + self.accrued
        else:
            return self.cleanprice(ytm)
        

            
bond = Bond("2004-03-10", "2013-01-04", 4.5)
print(f"Accrued interest: EUR {bond.accrued:.4f} per bond")

edsp_conversion_factor = bond.cleanprice(0.06) / 100
print(f"Conversion factor: {edsp_conversion_factor:.4f}")

edsp_conversion_factor = bond.cleanprice(0.06) / 100
print(f"Settlement amount: EUR {(116.80 * edsp_conversion_factor + bond.accrued) * 1000:,.2f}")

Accrued interest: EUR 0.8115 per bond
Conversion factor: 0.8993
Settlement amount: EUR 105,854.85


#### Task: Calculate the Implied Repo Rate.

The Future quote is 116,80 and the bonds spot price is 105,20 (clean). We are trading Settlement on 2004-02-09 that means 30 days before the delivery of the future. What is the conclusion if the only other deliverable bond has an implied repo rate of 1,80%?

In [16]:
bond_ctd = Bond("2004-02-09", "2013-01-04", 4.5)

future_settlement = 116.80 * edsp_conversion_factor + bond.accrued
dirty_price_today = 105.20 + bond_ctd.accrued
implied_repo = (future_settlement - dirty_price_today) / (dirty_price_today * 30/360)

print(f"Implied repo: {implied_repo*100:.2f}%")

Implied repo: 2.41%


#### Task: Calculate the forward price for the delivery date of the future.

We are still trading spot settlement on 2004-02-09. The 30 day money rate is 2%. 

In [17]:
financing_costs = (105.20 + bond_ctd.accrued) * 0.02 * 30/360

# Note that the carry portfolio has to hold the bond from 2004-02-09 until 2004-03-10
accrued_difference = (bond.accrued - bond_ctd.accrued)

forward_price = 105.20 + financing_costs - accrued_difference

print(f"Forward price: EUR {forward_price:.2f}")

Forward price: EUR 105.01


#### Task: Calculate the theoretical future price.

In [18]:
theoretical_future_price = forward_price / edsp_conversion_factor

print(f"Theoretical future price: EUR {theoretical_future_price:.2f}")

Theoretical future price: EUR 116.76


#### Task: Calculate the basis point hedge.

Calculate the basis point hedge for a nominal 10 million spot position of a 5% nine year bond, which currently has a YTM of 4,00%. You are hedging with the CTD from above (settlement 2004-02-09), which currently has a YTM of 3.7995%.

In [21]:
pv01_bond = (bond_price(0.04+0.0001, 0.05, 9) - bond_price(0.04, 0.05, 9)) * 100
print(f"PV01 of the bond: EUR {pv01_bond:.6f}")

pv01_ctd = (bond_ctd.cleanprice(0.037995 + 0.0001) - bond_ctd.cleanprice(0.037995))
print(f"PV01 of the CTD:  EUR {pv01_ctd:.6f}")

nominal_hedge = 10e+6 / 100000
hedge_ratio = nominal_hedge * edsp_conversion_factor * pv01_bond / pv01_ctd
print(f"Basis point hedge ratio: {hedge_ratio:.2f}")

PV01 of the bond: EUR -0.077706
PV01 of the CTD:  EUR -0.076693
Basis point hedge ratio: 91.12
